In [2]:
import os
import cv2
import numpy as np
import torch
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/isb-data/Dataset_clean/dataset.csv
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_163140.494-000015.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_160532.978-000046.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_161844.175-000042.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_164704.138-000040.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_163829.194-000018.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_162500.282-000060.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_163511.574-000012.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_162500.965-000061.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_163512.822-000015.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_164843.453-000037.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_163020.443-000041.jpg
/kaggle/input/isb-data/Dataset_clean/apple/2/20190729_163149.909-000034.jpg
/kaggle/input/isb-data/Dataset_clean/ap

In [3]:
# Set up the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Load the pre-trained Mask R-CNN model
model = maskrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [ ]:
classes = ['apple','banana','carrot']

In [ ]:
# Iterate through the relevant folders
for folder in ['apple', 'banana', 'carrot']:
    folder_path = os.path.join('/kaggle/input','isb-data', 'Dataset_clean', folder)
    for sub_folder in os.listdir(folder_path):
        sub_folder_path = os.path.join(folder_path, sub_folder)
        for file in os.listdir(sub_folder_path):
            if file.endswith('.jpg'):
                # Load the image
                img_path = os.path.join(sub_folder_path, file)
                img = Image.open(img_path)

                # Preprocess the image
                img_tensor = F.to_tensor(img).to(device)
                img_tensor = img_tensor.unsqueeze(0)

                # Run object detection
                with torch.no_grad():
                    outputs = model(img_tensor)

                # Save the output
                output_path = os.path.join(output_dir, f'{folder}_{sub_folder}_{file}')
                img_np = np.array(img)

                for i, box in enumerate(outputs[0]['boxes']):
                    x1, y1, x2, y2 = [int(coord.item()) for coord in box]
                    label_idx = outputs[0]['labels'][i].item()
                    label = classes[label_idx] if label_idx < len(classes) else 'Unknown'
                    score = outputs[0]['scores'][i].item()

                    if score >= 0.5:  # Only consider detections with confidence score >= 0.5
                        img_np = cv2.rectangle(img_np, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        img_np = cv2.putText(img_np, f'{label} ({score:.2f})', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)

                        # Add segmentation masks to the image
                        mask = outputs[0]['masks'][i, 0].mul(255).byte().cpu().numpy()
                        colored_mask = np.zeros_like(img_np, dtype=np.uint8)
                        colored_mask[:, :, 1] = mask  # Green channel
                        img_np = cv2.addWeighted(img_np, 1, colored_mask, 0.5, 0)

                cv2.imwrite(output_path, img_np)
